# Carga da Dimensão Tempo

Este notebook realiza a carga da dimensão tempo (dim_tempo).

## Imports

In [1]:
from spark_config import init_spark
from pyspark.sql import functions as F
from pyspark.sql.types import DateType, IntegerType, StringType, BooleanType, TimestampType
from datetime import datetime, timedelta
import pandas as pd

## Start Spark Session

In [ ]:
spark = init_spark("Carga dimensão tempo")


✅ Spark 3.5.7 iniciado com Hive local persistente!
📁 Warehouse: D:/Projetos/DataLake/spark-warehouse
📁 Metastore: D:/Projetos/DataLake/metastore_db



## Variables

In [ ]:
# Definir intervalo de datas
data_inicio = "2020-01-01"
data_fim = "2030-12-31"

# Nome da tabela
tabela_destino = "gold.dim_tempo"

# Feriados fixos brasileiros (formato: mês-dia)
feriados_fixos = [
    "01-01",  # Ano Novo
    "04-21",  # Tiradentes
    "05-01",  # Dia do Trabalho
    "09-07",  # Independência do Brasil
    "10-12",  # Nossa Senhora Aparecida
    "11-02",  # Finados
    "11-15",  # Proclamação da República
    "11-20",  # Consciência Negra
    "12-25",  # Natal
]

## Read Source Data

In [ ]:
# Criar lista de datas
start_date = datetime.strptime(data_inicio, "%Y-%m-%d")
end_date = datetime.strptime(data_fim, "%Y-%m-%d")
date_list = []

current_date = start_date
while current_date <= end_date:
    date_list.append((current_date,))
    current_date += timedelta(days=1)

[SUCESSO] Leitura da tabela staging em: D:/Projetos/Jornada_financas_pessoais/data/delta/silver/stg_cotacao_historica
Total de registros: 5126223


## Transform Data

In [ ]:
# Criar DataFrame inicial com as datas
df_datas = spark.createDataFrame(date_list, ["dt_dia"])

# Adicionar colunas básicas de data
df_tempo = df_datas.withColumn("nr_dia", dayofmonth(col("dt_dia"))) \
    .withColumn("nr_dia_ano", dayofyear(col("dt_dia"))) \
    .withColumn("nr_dia_semana", dayofweek(col("dt_dia"))) \
    .withColumn("nr_mes", month(col("dt_dia"))) \
    .withColumn("nr_ano", year(col("dt_dia")))

# Nome do dia da semana em português
df_tempo = df_tempo.withColumn(
    "nm_dia_semana",
    when(col("nr_dia_semana") == 1, "Domingo")
    .when(col("nr_dia_semana") == 2, "Segunda-feira")
    .when(col("nr_dia_semana") == 3, "Terça-feira")
    .when(col("nr_dia_semana") == 4, "Quarta-feira")
    .when(col("nr_dia_semana") == 5, "Quinta-feira")
    .when(col("nr_dia_semana") == 6, "Sexta-feira")
    .when(col("nr_dia_semana") == 7, "Sábado")
)

# Nome do mês em português
df_tempo = df_tempo.withColumn(
    "nm_mes",
    when(col("nr_mes") == 1, "Janeiro")
    .when(col("nr_mes") == 2, "Fevereiro")
    .when(col("nr_mes") == 3, "Março")
    .when(col("nr_mes") == 4, "Abril")
    .when(col("nr_mes") == 5, "Maio")
    .when(col("nr_mes") == 6, "Junho")
    .when(col("nr_mes") == 7, "Julho")
    .when(col("nr_mes") == 8, "Agosto")
    .when(col("nr_mes") == 9, "Setembro")
    .when(col("nr_mes") == 10, "Outubro")
    .when(col("nr_mes") == 11, "Novembro")
    .when(col("nr_mes") == 12, "Dezembro")
)

# Descrição mês e ano
df_tempo = df_tempo.withColumn(
    "ds_mes_ano",
    concat_ws(" ", col("nm_mes"), col("nr_ano").cast("string"))
)

# Período da semana (segunda a domingo)
df_tempo = df_tempo.withColumn(
    "segunda_feira_semana",
    when(col("nr_dia_semana") == 1, date_sub(col("dt_dia"), 6))
    .when(col("nr_dia_semana") == 2, col("dt_dia"))
    .when(col("nr_dia_semana") == 3, date_sub(col("dt_dia"), 1))
    .when(col("nr_dia_semana") == 4, date_sub(col("dt_dia"), 2))
    .when(col("nr_dia_semana") == 5, date_sub(col("dt_dia"), 3))
    .when(col("nr_dia_semana") == 6, date_sub(col("dt_dia"), 4))
    .when(col("nr_dia_semana") == 7, date_sub(col("dt_dia"), 5))
)

df_tempo = df_tempo.withColumn(
    "domingo_semana",
    when(col("nr_dia_semana") == 1, col("dt_dia"))
    .when(col("nr_dia_semana") == 2, date_sub(col("dt_dia"), -6))
    .when(col("nr_dia_semana") == 3, date_sub(col("dt_dia"), -5))
    .when(col("nr_dia_semana") == 4, date_sub(col("dt_dia"), -4))
    .when(col("nr_dia_semana") == 5, date_sub(col("dt_dia"), -3))
    .when(col("nr_dia_semana") == 6, date_sub(col("dt_dia"), -2))
    .when(col("nr_dia_semana") == 7, date_sub(col("dt_dia"), -1))
)

df_tempo = df_tempo.withColumn(
    "ds_periodo_semana",
    concat_ws(
        " a ",
        date_format(col("segunda_feira_semana"), "dd-MM-yyyy"),
        date_format(col("domingo_semana"), "dd-MM-yyyy")
    )
).drop("segunda_feira_semana", "domingo_semana")

# COMMAND ----------

# Bimestre
df_tempo = df_tempo.withColumn(
    "nr_bimestre",
    when(col("nr_mes").isin([1, 2]), 1)
    .when(col("nr_mes").isin([3, 4]), 2)
    .when(col("nr_mes").isin([5, 6]), 3)
    .when(col("nr_mes").isin([7, 8]), 4)
    .when(col("nr_mes").isin([9, 10]), 5)
    .when(col("nr_mes").isin([11, 12]), 6)
).withColumn(
    "ds_bimestre",
    concat_ws("", col("nr_bimestre").cast("string"), lit("º Bimestre"))
)

# Trimestre
df_tempo = df_tempo.withColumn(
    "nr_trimestre",
    when(col("nr_mes").isin([1, 2, 3]), 1)
    .when(col("nr_mes").isin([4, 5, 6]), 2)
    .when(col("nr_mes").isin([7, 8, 9]), 3)
    .when(col("nr_mes").isin([10, 11, 12]), 4)
).withColumn(
    "ds_trimestre",
    concat_ws("", col("nr_trimestre").cast("string"), lit("º Trimestre"))
)

# Quadrimestre
df_tempo = df_tempo.withColumn(
    "nr_quadrimestre",
    when(col("nr_mes").isin([1, 2, 3, 4]), 1)
    .when(col("nr_mes").isin([5, 6, 7, 8]), 2)
    .when(col("nr_mes").isin([9, 10, 11, 12]), 3)
).withColumn(
    "ds_quadrimestre",
    concat_ws("", col("nr_quadrimestre").cast("string"), lit("º Quadrimestre"))
)

# Semestre
df_tempo = df_tempo.withColumn(
    "nr_semeste",
    when(col("nr_mes").isin([1, 2, 3, 4, 5, 6]), 1)
    .when(col("nr_mes").isin([7, 8, 9, 10, 11, 12]), 2)
).withColumn(
    "ds_semestre",
    concat_ws("", col("nr_semeste").cast("string"), lit("º Semestre"))
)

# Último dia do mês
df_tempo = df_tempo.withColumn("dt_ultimo_dia_mes", last_day(col("dt_dia")))

# Criar coluna para verificar feriados fixos
feriados_fixos_formatados = [f"-{f}" for f in feriados_fixos]

df_tempo = df_tempo.withColumn(
    "mes_dia",
    date_format(col("dt_dia"), "-MM-dd")
)

# Flag de feriado fixo
feriado_fixo_condition = col("mes_dia").isin(feriados_fixos_formatados)

# Flag de feriado móvel
feriados_moveis_list = [datetime.strptime(d, "%Y-%m-%d").date() for d in feriados_moveis]
df_tempo = df_tempo.withColumn(
    "fl_feriado",
    when(feriado_fixo_condition, lit(True))
    .when(col("dt_dia").isin(feriados_moveis_list), lit(True))
    .otherwise(lit(False))
).drop("mes_dia")

# Flag de fim de semana
df_tempo = df_tempo.withColumn(
    "fl_fim_semana",
    when(col("nr_dia_semana").isin([1, 7]), lit(True)).otherwise(lit(False))
)

# Flag de dia útil (não é feriado e não é fim de semana)
df_tempo = df_tempo.withColumn(
    "fl_dia_util",
    when((col("fl_feriado") == False) & (col("fl_fim_semana") == False), lit(True))
    .otherwise(lit(False))
)

 Criar uma janela para calcular o último dia útil do mês
from pyspark.sql.window import Window

# Criar coluna ano_mes para agrupar
df_tempo = df_tempo.withColumn("ano_mes", date_format(col("dt_dia"), "yyyy-MM"))

# Ordenar por data descendente dentro de cada mês e filtrar dias úteis
windowSpec = Window.partitionBy("ano_mes").orderBy(col("dt_dia").desc())

from pyspark.sql.functions import row_number, max as max_

df_ultimo_util = df_tempo.filter(col("fl_dia_util") == True) \
    .withColumn("rn", row_number().over(windowSpec)) \
    .filter(col("rn") == 1) \
    .select(
        col("ano_mes"),
        col("dt_dia").alias("dt_ultimo_dia_util_mes_calc")
    )

# Juntar com o dataframe principal
df_tempo = df_tempo.join(df_ultimo_util, on="ano_mes", how="left")

df_tempo = df_tempo.withColumn(
    "dt_ultimo_dia_util_mes",
    col("dt_ultimo_dia_util_mes_calc")
).drop("ano_mes", "dt_ultimo_dia_util_mes_calc")

# Adicionar timestamp de inserção
df_tempo = df_tempo.withColumn("ts_insercao", current_timestamp())

In [ ]:
df_final = df_tempo.select(
    "dt_dia",
    "nr_dia",
    "nr_dia_ano",
    "nr_dia_semana",
    "nm_dia_semana",
    "ds_periodo_semana",
    "nr_mes",
    "nm_mes",
    "ds_mes_ano",
    "nr_bimestre",
    "ds_bimestre",
    "nr_trimestre",
    "ds_trimestre",
    "nr_quadrimestre",
    "ds_quadrimestre",
    "nr_semeste",
    "ds_semestre",
    "nr_ano",
    "dt_ultimo_dia_util_mes",
    "dt_ultimo_dia_mes",
    "fl_feriado",
    "fl_fim_semana",
    "fl_dia_util",
    "ts_insercao"
)

display(df_final.limit(100))

# Estatísticas básicas
print(f"Total de registros: {df_final.count()}")
print(f"Período: {data_inicio} até {data_fim}")
print(f"\nDistribuição de dias úteis:")
df_final.groupBy("fl_dia_util").count().show()


## Write Data

In [ ]:
# Modo de escrita: overwrite (sobrescrever) ou append (adicionar)
modo_escrita = "overwrite"

# Escrever na tabela Delta
df_final.write \
    .format("delta") \
    .mode(modo_escrita) \
    .saveAsTable(tabela_destino)

print(f"✅ Dados inseridos com sucesso na tabela {tabela_destino}")

✅ Merge realizado com sucesso
🆕 Inseridos: 2519
♻️ Atualizados: 0


## Stop Spark Session

In [7]:
# Encerra a SparkSession
spark.stop()